In [1]:
# required Python imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import funciones as f
import pandas as pd
from types import SimpleNamespace

from scipy.spatial.transform import Rotation as rot
from scipy.optimize import minimize

In [2]:
# STEP 1
database = SimpleNamespace(euler=f.explore_Euler_space(step=15))
database.euler.shape

(637, 3)

In [3]:
# Create a DataFrame
df = pd.DataFrame(database.euler, columns=['euler1', 'euler2', 'euler3'])
df

,euler1,euler2,euler3
0,0,0,0
1,0,0,15
2,0,0,30
3,0,0,45
4,0,0,60
...,...,...,...
632,90,180,30
633,90,180,45
634,90,180,60
635,90,180,75


In [4]:
def find_nearest(df, values):
    """find the index of the nearest value in
    a pandas dataframe"""
    indexes = []
    for value in values:
         indexes.append((np.abs(df - value)).idxmin())
    return indexes

In [5]:
# Estimate a mesh of values defining the transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(90, 50, 20), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

In [6]:
angles = np.arange(0, 100, 18)  # array([ 0, 18, 36, 54, 72, 90])

# initialize variables
T_vals = []
azi_vals = []

for euler in database.euler:  
    
    # rotate
    x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)
    
    # extract XY intersection
    xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)
    
    # get the indexes of specific angles
    indexes = find_nearest(xy_vectors['angles'], angles)
    
    # append values
    T_vals.append(xy_vectors.loc[indexes, ['T']].T.values.tolist()[0])
    azi_vals.append(xy_vectors.loc[indexes, ['angles']].T.values.tolist()[0])

# store values in database
database.T_values = np.array(T_vals)
database.azimuths = np.array(azi_vals) 
    

In [7]:
database

namespace(euler=array([[  0,   0,   0],
                       [  0,   0,  15],
                       [  0,   0,  30],
                       ...,
                       [ 90, 180,  60],
                       [ 90, 180,  75],
                       [ 90, 180,  90]]),
          T_values=array([[89.999538  , 86.1650861 , 76.13204083, 63.74754894, 53.76041284,
                           50.00017573],
                          [87.51597179, 89.89815474, 84.89892033, 74.18660111, 61.86880897,
                           52.66877195],
                          [80.25294895, 88.19456563, 89.59503116, 83.48169882, 72.1987119 ,
                           60.07250355],
                          ...,
                          [79.81730891, 67.67549844, 56.79902195, 50.4833652 , 51.65118401,
                           59.85467903],
                          [87.26735993, 78.01530806, 65.68967501, 55.01669544, 50.08889498,
                           52.79582799],
                          [89.9995

In [8]:
print(database.euler.shape)
print(database.T_values.shape)
print(database.azimuths.shape)

(637, 3)
(637, 6)
(637, 6)


In [9]:
print(f'euler angles: {np.around(database.euler[0], 1)}')
print(f'T values: {np.around(database.T_values[0], 1)}')
print(f'azimuth angles: {np.around(database.azimuths[0], 1)}')

euler angles: [0 0 0]
T values: [90.  86.2 76.1 63.7 53.8 50. ]
azimuth angles: [ 0.  18.  36.1 54.1 72.1 90.2]


In [10]:
print(f'euler angles: {np.around(database.euler[500], 1)}')
print(f'T values: {np.around(database.T_values[500], 1)}')
print(f'azimuth angles: {np.around(database.azimuths[500], 1)}')

euler angles: [75 90 45]
T values: [36.5 45.9 51.9 51.9 45.9 36.3]
azimuth angles: [ 0.3 17.9 35.9 54.1 72.1 90. ]
